---
title: 21天学通C++ Chapter13 类型转换运算符
tags: 小书匠,21天学通C++,static_cast,const_cast
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# 21天学通C++ Chapter13 类型转换运算符

## C 风格的类型转换

优点是书写简单并且直观。缺点是不会进行检查。

In [4]:
char s[] = {'h', 'e', 'l', 'l', 'o', '\0'};
int* intArray = (int*) s

## C++ 的四种类型转换符

- static_cast 用于编译时转换
- dynamic_cast 用于运行时转换
- const_cast 用于将 const 对象转换为非 const 对象
- reinterpret_cast 类型于 C 风格转换

In [28]:
#include <iostream>

class Base { };
class Derived: public Base {};

## static_cast

static_cast 在类型转换的时候会检查是否相关（也就是说有继承关系），主要用于 Downcast，upcast 不需要 static_cast 就可以直接转换。

- Upcast:  Derived -> Base 
- DownCast: Base -> Derived

### UpCast

In [29]:
Derived* pDerived = new Derived;
Base* pBase = pDerived; // 默认 upcast

也可以使用 static_cast

In [30]:
Base* pBase = static_cast<Base*>(pDerived);

### DownCast

In [31]:
Base* pBase = new Derived;
Derived* pDerived = static_cast<Derived*>(pBase);

当然，还可以使用 C 风格

In [32]:
Base* pBase = new Derived;
Derived* pDerived = (Derived*)pBase;

但是如果是一个没有继承关系的类进行转换，static_cast 会阻止这种行为，而 C 风格的类型转换不会报错

In [38]:
class Unrelated {};

Unrelated* pUnrelated = (Unrelated*)pBase
// static_cast 会检测出这种错误
// Unrelated* pUnrelated = static_cast<Unrelated*>(pBase) //  error: static_cast from 'Base *' to 'Unrelated *', which are not related by inheritance, is not allowed

## dynamic_cast

static_cast 只能静态检测，但是有可能有的错误是动态发生的。如下面的例子，其中试图将一个 Base 对象赋值给一个 Derived 指针。这在 static_cast 看来是正确的，因为 Base 和 Derived 是有继承关系的。

In [23]:
Base* pBase = new Base;
Derived* pDerived = static_cast<Derived*>(pBase);

但是，再 dynamic_cast 看来这是错误的，因为 Base 对象转换到 Derived 对象是缺少信息的。

In [39]:
Base* pBase = new Base;
// Derived* pDerived = dynamic_cast<Derived*>(pBase); // Base' is not polymorphic

## const_cast

In [40]:
#include <iostream>
class SimpleClass {
    public:
        void sayHello() {
            std::cout << "hello world!\n";
        }
    
}

In [41]:
void myFunc(const SimpleClass& simple) 
{
    simple.sayHello(); // error: member function 'sayHello' not viable: 'this' argument has type 'const SimpleClass', but function is not marked const
}

input_line_51:3:5: error: member function 'sayHello' not viable: 'this' argument has type 'const __cling_N541::SimpleClass', but function is not marked const
    simple.sayHello();
    ^~~~~~
input_line_50:3:14: note: 'sayHello' declared here
        void sayHello() {
             ^


Interpreter Error: 

上面的例子中，sayHello 函数本身不修改对象，因此在 myFunc 中使用一个 const 引用是没有问题的。但是 SimpleClass 源码中由于 sayHello 没有标志为 const，因此导致 myFunc 的调用会报错。

如果 SimpleClass 我们维护的代码，我们可以通过给 sayHello 添加 const 标志来解决。但是很多情况下我们都是使用第三方库，此时我们就只能使用 const_cast 来将 simple 变成非 const 引用了。

In [48]:
void myFunc(const SimpleClass& simple) 
{
    SimpleClass& nonConstSimple = const_cast<SimpleClass&>(simple);
    nonConstSimple.sayHello(); // error: member function 'sayHello' not viable: 'this' argument has type 'const SimpleClass', but function is not marked const
}

# References

- 21天学通C++ 第8版 第13章 类型转换运算符